# Pytorch TensorBoard

[Original video](https://youtu.be/RLqsxWaQdHE)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard

In [ ]:
# Create convolutional neural network (CNN)
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10, size=28):
        super(CNN, self).__init__()
        out_channels=16
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8,
                               kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=out_channels,
                               kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(int(out_channels*(size/4)*(size/4)), num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [ ]:
# Hyperparameters
in_channel = 1
num_classes = 10
# learning_rate = 1e-4
# batch_size = 1024
num_epochs = 1

In [ ]:
# Check if it runs correctly
model = CNN()
x = torch.randn(32, 1, 28, 28)

# Run model on the input and print the shape
print(model(x).shape)

torch.Size([32, 10])


In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# For the error: HTTPError: HTTP Error 403: Forbidden
# StackOverflow: https://stackoverflow.com/a/66461122/7550928
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model, test=True):
    if test:
        print('Checking accuracy on test data', end='')
    else:
        print('Checking accuracy on training data', end='')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    acc = float(num_correct) / float(num_samples) * 100
    #print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

    model.train()
    return acc

In [ ]:
# Train network
def train():
    # Initialize network
    model = CNN().to(device)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
    
    # TensorBoard writer
    writer = SummaryWriter(f'runs/MNIST/MiniBatchSize {batch_size} LR {learning_rate}')

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

    step = 0
    model.train()
    for epoch in range(num_epochs):
        losses = []

        for batch_idx, (data, targets) in enumerate(train_loader):
            # Get data to Cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # Forward
            scores = model(data)  # shape 64x10
            loss = criterion(scores, targets)
            losses.append(loss.item())

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()  # gradient descent or adam step

            # Calculate running training accuracy
            _, predictions = scores.max(dim=1)
            num_correct = (predictions == targets).sum()
            train_acc = float(num_correct / float(data.shape[0]))

            # Plot things to TensorBoard
            img_grid = torchvision.utils.make_grid(data)
            writer.add_image('mnist_images', img_grid)
            writer.add_histogram('fc1', model.fc1.weight)
            writer.add_scalar('Training loss', loss, global_step=step)
            writer.add_scalar('Training accuracy', train_acc, global_step=step)
            # if batch_idx % 10 == 0:
            #     print(batch_idx)
            if batch_idx == 230:
                features = data.reshape(data.shape[0], -1)
                class_labels = [classes[label] for label in predictions]
                writer.add_embedding(features,
                                     metadata=class_labels,
                                     label_img=data,
                                     global_step=batch_idx)
            step += 1
        
        mean_loss = sum(losses)/len(losses)
        acc = check_accuracy(test_loader, model)
        msg = (f'\rLoss at epoch {epoch} is {mean_loss:.5f}. '
               f'bs {batch_size} lr {learning_rate} '
               f'Accuracy is {acc:.2f}')
        print(msg)

        writer.add_hparams(
                {"lr": learning_rate, "bsize": batch_size},
                {
                    "accuracy": acc,
                    "loss": sum(losses) / len(losses),
                },
            )

[TensorBoard in notebooks](https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb#scrollTo=IlDz2oXBgnZ9)

In [ ]:
# Remove previous runs if necessary
if os.path.exists('runs'):
    !rm -rf runs

# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# Start TensorBoard before training to monitor it in progress
%tensorboard --logdir runs

# Reload TensorBoard
%reload_ext tensorboard

In [ ]:
# To fix the error, because PyTorch and TensorFlow are installed both:
# AttributeError: module 'tensorflow._api.v2.io.gfile' has no attribute 'get_filesystem'
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [ ]:
# Check various hyperparameters
# batch_sizes = [4, 64, 1024]
batch_sizes = [256]
# learning_rates = [0.1, 0.01, 0.001, 0.0001]
learning_rates = [0.001]
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        train()

Loss at epoch 0 is 0.69204. bs 256 lr 0.001 Accuracy is 93.39
